In [57]:
import pyPI as pi
import numpy as np
from scipy.linalg import eig
import sympy as sp
from scipy.optimize import minimize

In [58]:
# Vectores de entrada y salida
E_in = pi.polarization_basis_set('L')
S = pi.jones_matrix(np.pi/8, np.pi/16)
E_out = S @ E_in
S

array([[0.99710284-0.01456502j, 0.01456502+0.0732233j ],
       [0.01456502+0.0732233j , 0.9267767 -0.36811841j]])

In [59]:
# Eigenvalue and eigenvector calculation (Principal Component Analysis)
eigenvalues, eigenvectors = eig(S)

# Determine alpha (angle) from the dominant eigenvector
alpha = np.arctan2(np.real(eigenvectors[1, 0]), np.real(eigenvectors[0, 0]))

print(f"α (dominant direction): {alpha / np.pi:.4f}π\n")

α (dominant direction): 0.0625π



In [60]:
# Ejemplo de uso con vectores específicos:
x_1 = E_in
x_2 = pi.polarization_basis_set('H')

b_1 = E_out
b_2 = S @ x_2

x_vectors = [x_1, x_2]
b_vectors = [b_1, b_2]

A, alpha_calculated = pi.determine_A(x_vectors, b_vectors, p=True)

Determined matrix A:
[[0.99710284-0.01456502j 0.01456502+0.0732233j ]
 [0.01456502+0.0732233j  0.9267767 -0.36811841j]] 

α (dominant direction): 0.0625π



In [61]:
# Matriz por minimizacion
optimal_delta_chi, optimal_alpha = pi.birefringence_by_minimization(E_in, E_out, p = True)

χ: 0.1250π
α: 0.0625π


In [62]:
print(f"α original: {alpha / np.pi:.4f}π\n")
print(f"α calculated: {alpha_calculated / np.pi:.4f}π\n")
print(f"α by minimization: {optimal_alpha / np.pi:.4f}π\n")

α original: 0.0625π

α calculated: 0.0625π

α by minimization: 0.0625π



In [ ]:
import sympy as sp

# Definición de variables simbólicas
delta_chi = sp.Symbol('delta_chi', real=True)  # Incógnita
alpha = sp.Symbol('alpha', real=True)
A_isx, A_isy = sp.symbols('A_isx A_isy', real=True)
delta_phi_is = sp.Symbol('delta_phi_is', real=True)
A_sx, phi_sx, A_sy, phi_sy = sp.symbols('A_sx phi_sx A_sy phi_sy', real=True)

# Expresiones de las ecuaciones
S_xx = (sp.cos(alpha)**2 + sp.exp(-1j * delta_chi) * sp.sin(alpha)**2) * A_isx
S_xy = ((1 - sp.exp(-1j * delta_chi)) * sp.cos(alpha) * sp.sin(alpha)) * A_isy * sp.exp(1j * delta_phi_is)

S_yx = ((1 - sp.exp(-1j * delta_chi)) * sp.cos(alpha) * sp.sin(alpha)) * A_isx
S_yy = (sp.sin(alpha)**2 + sp.exp(-1j * delta_chi) * sp.cos(alpha)**2) * A_isy * sp.exp(1j * delta_phi_is)

# Ecuaciones completas
eq1 = sp.Eq(A_sx * sp.exp(1j * phi_sx), S_xx + S_xy)
eq2 = sp.Eq(A_sy * sp.exp(1j * phi_sy), S_yx + S_yy)

# Asignación de valores específicos
values = {
    alpha: alpha_calculated,
    A_isx: np.abs(E_in[0,0]),
    A_isy: np.abs(E_in[1,0]),
    delta_phi_is: np.angle(E_in[1,0]),
    A_sx: np.abs(E_out[0,0]),
    phi_sx: np.angle(E_out[0,0]),
    A_sy: np.abs(E_out[1,0]),
    phi_sy: np.angle(E_out[1,0])
}

# Resolviendo para delta_chi
solution = sp.solve([eq1.subs(values), eq2.subs(values)], delta_chi)

# Mostrar la solución
print("Solución para delta_chi:", solution)


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'dict'

In [64]:
values

{alpha: 0.19634954084936196,
 A_isx: 0.7071067811865475,
 A_isy: 0.7071067811865475,
 delta_phi_is: 1.5707963267948966,
 A_sx: 0.6532814824381882,
 phi_sx: 2.655399735964416e-18,
 A_sy: 0.757115119848602,
 phi_sy: 1.205306570826704}